In [1]:
import sys, time
sys.argv = ["main", "../Input/data_webdir_leb.pkl", 2, 4, "Models/temp2_4.pt", "Output/test2_4.pkl"]

In [7]:
import os, itertools, time, pickle, operator, random
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re, sys
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

f = open(sys.argv[1], "rb")
data_small, data_large, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts = pickle.load(f)
max_paths = int(sys.argv[2])
max_pathlen = int(sys.argv[3])
max_types = 2
flatten = lambda l: [item for sublist in l for item in sublist]

direct_inputs, direct_targets = [], []

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a,b)

all_fn, all_fp = [], []

threshold_results = {}

def test():
    global batch_size, test_data_t, test_data_f, model, optimizer, all_metrics, direct_inputs, direct_targets, threshold_results, emb_indexer_inv, emb_vals
    all_results = OrderedDict()    
    direct_inputs, direct_targets = [], []
    with torch.no_grad():
        all_pred = []
        
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)

        inputs_pos, nodes_pos, targets_pos = generate_input(test_data_t, 1)
        inputs_neg, nodes_neg, targets_neg = generate_input(test_data_f, 0)

        inputs_all = list(inputs_pos) + list(inputs_neg)
        targets_all = list(targets_pos) + list(targets_neg)
        nodes_all = list(nodes_pos) + list(nodes_neg)
        
        all_inp = list(zip(inputs_all, targets_all, nodes_all))
        all_inp_shuffled = random.sample(all_inp, len(all_inp))
        inputs_all, targets_all, nodes_all = list(zip(*all_inp_shuffled))

        batch_size = min(batch_size, len(inputs_all))
        num_batches = int(ceil(len(inputs_all)/batch_size))
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            inputs = np.array(to_feature(inputs_all[batch_start: batch_end]))
            targets = np.array(targets_all[batch_start: batch_end])
            nodes = np.array(nodes_all[batch_start: batch_end])
            
            inp_elems = torch.LongTensor(inputs).to(device)
            node_elems = torch.LongTensor(nodes).to(device)
            targ_elems = torch.DoubleTensor(targets)

            outputs = model(node_elems, inp_elems)
            outputs = [el.item() for el in outputs]
            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[nodes[idx][0]]
                ent2 = emb_indexer_inv[nodes[idx][1]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (round(pred_elem, 3), targets[idx])
        
        direct_targets = [True if el else False for el in direct_targets]
        
        print ("Len (direct inputs): ", len(direct_inputs))
        for idx, direct_input in enumerate(direct_inputs):
            ent1 = emb_indexer_inv[direct_input[0]]
            ent2 = emb_indexer_inv[direct_input[1]]
            sim = cos_sim(emb_vals[direct_input[0]], emb_vals[direct_input[1]])
            all_results[(ent1, ent2)] = (round(sim, 3), direct_targets[idx])
    return (test_data_t, all_results)

def optimize_threshold(emb_indexer_inv, emb_vals):
    global batch_size, val_data_t, val_data_f, model, optimizer, gt_mappings, all_metrics, direct_inputs, direct_targets, threshold_results
    all_results = OrderedDict()
    direct_inputs, direct_targets = [], []
    with torch.no_grad():
        all_pred = []
        
        np.random.shuffle(val_data_t)
        np.random.shuffle(val_data_f)

        inputs_pos, nodes_pos, targets_pos = generate_input(val_data_t, 1)
        inputs_neg, nodes_neg, targets_neg = generate_input(val_data_f, 0)

        inputs_all = list(inputs_pos) + list(inputs_neg)
        targets_all = list(targets_pos) + list(targets_neg)
        nodes_all = list(nodes_pos) + list(nodes_neg)
        
        all_inp = list(zip(inputs_all, targets_all, nodes_all))
        all_inp_shuffled = random.sample(all_inp, len(all_inp))
        inputs_all, targets_all, nodes_all = list(zip(*all_inp_shuffled))

        batch_size = min(batch_size, len(inputs_all))
        num_batches = int(ceil(len(inputs_all)/batch_size))
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            inputs = np.array(to_feature(inputs_all[batch_start: batch_end]))
            targets = np.array(targets_all[batch_start: batch_end])
            nodes = np.array(nodes_all[batch_start: batch_end])
            
            inp_elems = torch.LongTensor(inputs).to(device)
            node_elems = torch.LongTensor(nodes).to(device)
            targ_elems = torch.DoubleTensor(targets)

            outputs = model(node_elems, inp_elems)
            outputs = [el.item() for el in outputs]
            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[nodes[idx][0]]
                ent2 = emb_indexer_inv[nodes[idx][1]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (round(pred_elem, 3), targets[idx])
        
        direct_targets = [True if el else False for el in direct_targets]
        
        print ("Len (direct inputs): ", len(direct_inputs))
        for idx, direct_input in enumerate(direct_inputs):
            ent1 = emb_indexer_inv[direct_input[0]]
            ent2 = emb_indexer_inv[direct_input[1]]
            sim = cos_sim(emb_vals[direct_input[0]], emb_vals[direct_input[1]])
            all_results[(ent1, ent2)] = (round(sim, 3), direct_targets[idx])
        
        low_threshold = round(np.min([el[0] for el in all_results.values()]) - 0.02, 3)
        high_threshold = round(np.max([el[0] for el in all_results.values()]) + 0.02, 3)
        threshold = low_threshold
        step = 0.001
        print (low_threshold, high_threshold)
        while threshold < high_threshold:
            threshold = round(threshold, 3)
#             print (threshold)
            res = []
            for i,key in enumerate(all_results):
                if all_results[key][0] > threshold:
                    res.append(key)
            fn_list = [(key, all_results[key][0]) for key in val_data_t if key not in set(res)]
            fp_list = [(elem, all_results[elem][0]) for elem in res if not all_results[elem][1]]
            tp_list = [(elem, all_results[elem][0]) for elem in res if all_results[elem][1]]
            
            tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
            exception = False
            
            try:
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1score = 2 * precision * recall / (precision + recall)
                f2score = 5 * precision * recall / (4 * precision + recall)
                f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
            except Exception as e:
                print (e)
                exception = True
                step = 0.001
                threshold += step
                continue
            # print ("Threshold: ", threshold, precision, recall, f1score, f2score, f0_5score)
            if threshold in threshold_results:
                #print (threshold, len(threshold_results[threshold]))
                threshold_results[threshold].append([precision, recall, f1score, f2score, f0_5score])
            else:
                threshold_results[threshold] = [[precision, recall, f1score, f2score, f0_5score]]
            threshold += step
        
def calculate_performance():
    global final_results
    all_metrics, all_fn, all_fp = [], [], []
    for (test_data_t, all_results) in final_results:
        res = []
        for i,key in enumerate(all_results):
            if all_results[key][0] > threshold:
                res.append(key)
        s = set(res)
        fn_list = [(key, all_results[key][0]) for key in test_data_t if key not in s]
        fp_list = [(elem, all_results[elem][0]) for elem in res if not all_results[elem][1]]
        tp_list = [(elem, all_results[elem][0]) for elem in res if all_results[elem][1]]
        tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
        
        try:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1score = 2 * precision * recall / (precision + recall)
            f2score = 5 * precision * recall / (4 * precision + recall)
            f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
        except Exception as e:
            print (e)
            continue
        all_fn.extend(fn_list)
        all_fp.extend(fp_list)
        all_metrics.append((precision, recall, f1score, f2score, f0_5score))
    return all_metrics, all_fn, all_fp

def masked_softmax(inp):
    inp = inp.double()
    mask = ((inp != 0).double() - 1) * 9999  # for -inf
    return (inp + mask).softmax(dim=-1)

class SiameseNetwork(nn.Module):
    def __init__(self, emb_vals, threshold=0.9):
        super().__init__() 
        
        self.n_neighbours = max_types
        self.max_paths = max_paths
        self.max_pathlen = max_pathlen
        self.embedding_dim = np.array(emb_vals).shape[1]
        
        self.name_embedding = nn.Embedding(len(emb_vals), self.embedding_dim)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
        self.name_embedding.weight.requires_grad = False
        
        self.threshold = nn.Parameter(torch.DoubleTensor([threshold]))
        self.threshold.requires_grad = False
        
        self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
        self.output = nn.Linear(2*self.embedding_dim, 300)
        
        self.v = nn.Parameter(torch.DoubleTensor([1/(self.max_pathlen) for i in range(self.max_pathlen)]))
        self.w_rootpath = nn.Parameter(torch.DoubleTensor([0.5]))
 
    def forward(self, nodes, features):
        '''
        Arguments:
            - nodes: batch_size * 2
            - features: batch_size * 2 * 4 * max_paths * max_pathlen
        '''
        results = []
        nodes = nodes.permute(1,0) # 2 * batch_size
        features = features.permute(1,0,2,3,4) # 2 * batch_size * 4 * max_paths * max_pathlen
        for i in range(2):
            node_emb = self.name_embedding(nodes[i]) # batch_size * 512
            feature_emb = self.name_embedding(features[i]) #  batch_size * 4 * max_paths * max_pathlen * 512
            
            feature_emb_reshaped = feature_emb.permute(0,4,1,2,3).reshape(-1, self.embedding_dim, self.n_neighbours * self.max_paths * self.max_pathlen)
            path_weights = torch.bmm(node_emb[:, None, :], feature_emb_reshaped)
            path_weights = path_weights.squeeze(1).reshape(-1, self.n_neighbours, self.max_paths, self.max_pathlen)
            path_weights = torch.sum(path_weights, dim=-1)
            best_path_indices = torch.max(path_weights, dim=-1)[1][(..., ) + (None, ) * 3]
            best_path_indices = best_path_indices.expand(-1, -1, -1, self.max_pathlen,  self.embedding_dim)
            best_path = torch.gather(feature_emb, 2, best_path_indices).squeeze(2) # batch_size * 4 * max_pathlen * 512
            # Another way: 
            # path_weights = masked_softmax(path_weights)
            # best_path = torch.sum(path_weights[:, :, :, None, None] * feature_emb, dim=2)

            best_path_reshaped = best_path.permute(0,3,1,2).reshape(-1, self.embedding_dim, self.n_neighbours * self.max_pathlen)
            node_weights = torch.bmm(node_emb.unsqueeze(1), best_path_reshaped) # batch_size * 4 * max_pathlen
            node_weights = masked_softmax(node_weights.squeeze(1).reshape(-1, self.n_neighbours, self.max_pathlen)) # batch_size * 4 * max_pathlen
            attended_path = node_weights.unsqueeze(-1) * best_path # batch_size * 4 * max_pathlen * 512

            distance_weighted_path = torch.sum((self.v[None,None,:,None] * attended_path), dim=2) # batch_size * 4 * 512

            self.w_children = (1-self.w_rootpath)
            context_emb = self.w_rootpath * distance_weighted_path[:,0,:] \
                        + self.w_children * distance_weighted_path[:,1,:]

            contextual_node_emb = torch.cat((node_emb, context_emb), dim=1)
            output_node_emb = self.output(contextual_node_emb)
            results.append(output_node_emb)
        sim = self.cosine_sim_layer(results[0], results[1])
        return sim

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

def generate_data_neighbourless(elem_tuple):
    return [emb_indexer[elem] for elem in elem_tuple]

def embedify(seq):
    for item in seq:
        if isinstance(item, list):
            yield list(embedify(item))
        else:
            yield emb_indexer[item]

def generate_data(elem_tuple):
    return list(embedify([neighbours_dicts[elem] for elem in elem_tuple]))

def to_feature(inputs):
    inputs_lenpadded = [[[[path[:max_pathlen] + [0 for i in range(max_pathlen -len(path[:max_pathlen]))]
                                    for path in nbr_type[:max_paths]]
                                for nbr_type in ent[:max_types]]
                            for ent in elem]
                        for elem in inputs]
    inputs_pathpadded = [[[nbr_type + [[0 for j in range(max_pathlen)]
                             for i in range(max_paths - len(nbr_type))]
                            for nbr_type in ent] for ent in elem]
                        for elem in inputs_lenpadded]
    return inputs_pathpadded

def generate_input(elems, target):
    inputs, targets, nodes = [], [], []
    global direct_inputs, direct_targets
    for elem in list(elems):
        try:
            inputs.append(generate_data(elem))
            nodes.append(generate_data_neighbourless(elem))
            targets.append(target)
        except:
            direct_inputs.append(generate_data_neighbourless(elem))
            direct_targets.append(target)
    return inputs, nodes, targets

print("Number of neighbours: " + str(sys.argv[1]))

def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])

torch.set_default_dtype(torch.float64)


all_metrics = []
final_results = []

lr = 0.001
num_epochs = 1
weight_decay = 0.001
batch_size = 32
dropout = 0.3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_t = {elem: data_large[elem] for elem in data_large if data_large[elem]}
data_f = {elem: data_large[elem] for elem in data_large if not data_large[elem]}

train_data_t = list(data_t.keys())
train_data_f = list(data_f.keys())
np.random.shuffle(train_data_f)
train_data_f = train_data_f[:150000]

train_data_t = np.repeat(train_data_t, ceil(len(train_data_f)/len(train_data_t)), axis=0)
train_data_t = train_data_t[:len(train_data_f)].tolist()

model = SiameseNetwork(emb_vals).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

for epoch in range(num_epochs):
    inputs_pos, nodes_pos, targets_pos = generate_input(train_data_t, 1)
    inputs_neg, nodes_neg, targets_neg = generate_input(train_data_f, 0)
    inputs_all = list(inputs_pos) + list(inputs_neg)
    targets_all = list(targets_pos) + list(targets_neg)
    nodes_all = list(nodes_pos) + list(nodes_neg)
    
    all_inp = list(zip(inputs_all, targets_all, nodes_all))
    all_inp_shuffled = random.sample(all_inp, len(all_inp))
    inputs_all, targets_all, nodes_all = list(zip(*all_inp_shuffled[:10]))

    batch_size = min(batch_size, len(inputs_all))
    num_batches = int(ceil(len(inputs_all)/batch_size))

    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx+1) * batch_size
        
        inputs = np.array(to_feature(inputs_all[batch_start: batch_end]))
        targets = np.array(targets_all[batch_start: batch_end])
        nodes = np.array(nodes_all[batch_start: batch_end])
        
        inp_elems = torch.LongTensor(inputs).to(device)
        targ_elems = torch.DoubleTensor(targets).to(device)
        node_elems = torch.LongTensor(nodes).to(device)

        optimizer.zero_grad()
        outputs = model(node_elems, inp_elems)
        loss = F.mse_loss(outputs, targ_elems)
        loss.backward()
        optimizer.step()

        if batch_idx%5000 == 0:
            print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

torch.save(model.state_dict(), sys.argv[4])

for i in range(5):
    data_t = {elem: data_small[elem] for elem in data_small if data_small[elem]}
    data_f = {elem: data_small[elem] for elem in data_small if not data_small[elem]}

    data_t_items = list(data_t.keys())
    data_f_items = list(data_f.keys())

    test_data_t = data_t_items[int((0.2*i)*len(data_t)):int((0.2*i + 0.2)*len(data_t))]
    test_data_f = data_f_items[int((0.2*i)*len(data_f)):int((0.2*i + 0.2)*len(data_f))]

    train_data_tt = data_t_items[:int(0.2*i*len(data_t))] + data_t_items[int(0.2*(i+1)*len(data_t)):]
    train_data_ff = data_f_items[:int(0.2*i*len(data_f))] + data_f_items[int(0.2*(i+1)*len(data_f)):]

    np.random.shuffle(train_data_tt)
    np.random.shuffle(train_data_ff)

    val_data_t = train_data_tt[:int(0.1*len(data_t))]
    val_data_f = train_data_ff[:int(0.1*len(data_f))]

    train_data_t = train_data_tt[int(0.1*len(data_t)):]
    train_data_f = train_data_ff[int(0.1*len(data_f)):]
    np.random.shuffle(train_data_f)
    train_data_f = train_data_f[:150000]

    print ("Training size:", len(train_data_t) + len(train_data_f) , "Val size:", len(val_data_t) + len(val_data_f))

    train_data_t = np.repeat(train_data_t, ceil(len(train_data_f)/len(train_data_t)), axis=0)
    train_data_t = train_data_t[:len(train_data_f)].tolist()
    
    model = SiameseNetwork(emb_vals).to(device)
    pretrained_dict = torch.load(sys.argv[4])
    model_dict = model.state_dict()

    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k!="name_embedding.weight"}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(num_epochs):
        inputs_pos, nodes_pos, targets_pos = generate_input(train_data_t, 1)
        inputs_neg, nodes_neg, targets_neg = generate_input(train_data_f, 0)
        inputs_all = list(inputs_pos) + list(inputs_neg)
        targets_all = list(targets_pos) + list(targets_neg)
        nodes_all = list(nodes_pos) + list(nodes_neg)
        
        all_inp = list(zip(inputs_all, targets_all, nodes_all))
        all_inp_shuffled = random.sample(all_inp, len(all_inp))
        inputs_all, targets_all, nodes_all = list(zip(*all_inp_shuffled))[:10]

        batch_size = min(batch_size, len(inputs_all))
        num_batches = int(ceil(len(inputs_all)/batch_size))

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size
            
            inputs = np.array(to_feature(inputs_all[batch_start: batch_end]))
            targets = np.array(targets_all[batch_start: batch_end])
            nodes = np.array(nodes_all[batch_start: batch_end])
            
            inp_elems = torch.LongTensor(inputs).to(device)
            targ_elems = torch.DoubleTensor(targets).to(device)
            node_elems = torch.LongTensor(nodes).to(device)

            optimizer.zero_grad()
            outputs = model(node_elems, inp_elems)
            loss = F.mse_loss(outputs, targ_elems)
            loss.backward()
            optimizer.step()

            if batch_idx%5000 == 0:
                print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

    model.eval()
    
    optimize_threshold(emb_indexer_inv, emb_vals)

    sys.stdout.flush()

    final_results.append(test())

threshold_results_mean = {el: np.mean(threshold_results[el], axis=0) for el in threshold_results}    
threshold = max(threshold_results_mean.keys(), key=(lambda key: threshold_results_mean[key][2]))

all_metrics, all_fn, all_fp = calculate_performance()

os.remove(sys.argv[4])

print ("Final Results: " + str(np.mean(all_metrics, axis=0)))
print ("Threshold: ", threshold)
pickle.dump([final_results, threshold_results], open(sys.argv[-1], "wb"))

Number of neighbours: ../Input/data_webdir_leb.pkl
Epoch: 0 Idx: 0 Loss: 0.21420708574637012
Training size: 2270 Val size: 323
Epoch: 0 Idx: 0 Loss: 0.24104197013788822
Len (direct inputs):  8
-0.439 1.004
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
Len (direct inputs):  17
Training size: 2269 Val size: 323
Epoch: 0 Idx: 0 Loss: 0.14860147830078327
Len (direct inputs):  5
-0.555 1.02
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by

In [13]:
emb_indexer

{'<UNK>': 0,
 'web_lebensmittel#Verzeichnis_Einkaufen-Sparen_Nahrungs-Genussmittel_Supermarkt': 1,
 'google_lebensmittel#World_Deutsch_Online-Shops_Essen-und-Trinken_Getraenke_Wein_Geschenke': 2,
 'google_lebensmittel#World_Deutsch_Online-Shops_Essen-und-Trinken_Getraenke_Wein_Europaeisch_Deutsch_Ahr': 3,
 'web_lebensmittel#Verzeichnis_Einkaufen-Sparen_Nahrungs-Genussmittel_Kraeuter-Gewuerze': 4,
 'google_lebensmittel#World_Deutsch_Online-Shops_Essen-und-Trinken_Honig': 5,
 'google_lebensmittel#World_Deutsch_Online-Shops_Essen-und-Trinken_Getraenke_Wein_Europaeisch_Deutsch_Pfalz': 6,
 'google_lebensmittel#World_Deutsch_Online-Shops_Essen-und-Trinken_Getraenke_Wein_Europaeisch_Deutsch_Nahe': 7,
 'web_lebensmittel#Verzeichnis_Einkaufen-Sparen_Nahrungs-Genussmittel_Getraenke': 8,
 'web_lebensmittel#Verzeichnis_Einkaufen-Sparen_Nahrungs-Genussmittel_Getraenke_Tee-Kaffee': 9,
 'google_lebensmittel#World_Deutsch_Online-Shops_Essen-und-Trinken_Regionale-Spezialitaeten_Europaeisch_Oesterreichi

In [9]:
import pickle

def calculate_performance():
    global final_results
    all_metrics, all_fn, all_fp = [], [], []
    for (test_onto, test_data_t, all_results) in final_results:
        res = []
        for i,key in enumerate(all_results):
            if all_results[key][0] > threshold:
                res.append(key)
        fn_list = [(key, all_results[key][0]) for key in test_data_t if key not in set(res)]
        fp_list = [(elem, all_results[elem][0]) for elem in res if not all_results[elem][1]]
        tp_list = [(elem, all_results[elem][0]) for elem in res if all_results[elem][1]]
        tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
        
        try:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
            f1score = 2 * precision * recall / (precision + recall)
            f2score = 5 * precision * recall / (4 * precision + recall)
            f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
        except Exception as e:
            print (e)
            continue
        print ("Performance for", test_onto, "is :", (precision, recall, f1score, f2score, f0_5score))
        all_fn.extend(fn_list)
        all_fp.extend(fp_list)
        all_metrics.append((precision, recall, f1score, f2score, f0_5score))
    return all_metrics, all_fn, all_fp



final_results1, threshold_results1 = pickle.load(open("Output/conf6_2_part1.pkl", "rb"))
final_results2, threshold_results2 = pickle.load(open("Output/conf6_2_part2.pkl", "rb"))
threshold_results = {}
for key in threshold_results1:
    threshold_results[key] = threshold_results1[key]
    if key in threshold_results2:
        threshold_results[key].extend(threshold_results2[key])
    


In [17]:
threshold_results_mean = {el: np.mean(threshold_results[el], axis=0) for el in threshold_results}    
threshold = max(threshold_results_mean.keys(), key=(lambda key: threshold_results_mean[key][2]))
final_results = final_results1 + final_results2

In [19]:
all_metrics, all_fn, all_fp = calculate_performance()
print ("Final Results: " + str(np.mean(all_metrics, axis=0)))
print ("Threshold: ", threshold)

Performance for [('confOf', 'sigkdd')] is : (0.8333333333333334, 0.7142857142857143, 0.7692307692307692, 0.7352941176470589, 0.8064516129032258)
Performance for [('iasted', 'sigkdd')] is : (0.8, 0.8, 0.8000000000000002, 0.8, 0.8)
Performance for [('cmt', 'ekaw')] is : (0.625, 0.45454545454545453, 0.5263157894736842, 0.4807692307692307, 0.5813953488372092)
Performance for [('conference', 'iasted')] is : (0.6666666666666666, 0.2857142857142857, 0.4, 0.3225806451612903, 0.5263157894736842)
Performance for [('edas', 'sigkdd')] is : (0.7, 0.4666666666666667, 0.56, 0.5, 0.6363636363636365)
Performance for [('ekaw', 'iasted')] is : (1.0, 0.6, 0.7499999999999999, 0.6521739130434783, 0.8823529411764706)
Final Results: [0.77083333 0.55353535 0.63425776 0.58180298 0.70547989]
Threshold:  0.934


In [26]:
len(pickle.load(open("Input/data_lebensmittel.pkl", "rb"))[0])

3240

In [ ]:
arr = [('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/google.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/web.owl'),
 ('german_datasets_copy/webdirectory/dmoz.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl'),
 ('german_datasets_copy/webdirectory/google.owl',
  'german_datasets_copy/webdirectory/web.owl'),
 ('german_datasets_copy/webdirectory/google.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl'),
 ('german_datasets_copy/webdirectory/web.owl',
  'german_datasets_copy/webdirectory/yahoo.small.owl')]

for ont_pair in arr:
    print (ont_pair)
    a, b, c = ont_pair[0], ont_pair[1], ont_pair[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() + "-" + ont_pair[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    !rm -rf $c
    os.mkdir(c)
    java_command = "java -jar logmap-matcher/target/logmap-matcher-4.0.jar MATCHER file:" +  os.path.abspath(a) + \
                     " file:" + os.path.abspath(b) + " " + "/data/Vivek/IBM/IBM-Internship/" + c + "/ false"
    process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
pred_logmap = [[el.split("/")[-1] for el in l.split("\t")[:-1]] for l in open(os.path.abspath(c + "/") + "/logmap2_mappings.tsv",  "r").read().split("\n")[:-1] if not l.startswith("Optional")]
gt_mappings = []
for elem in pred_logmap:
    gt_mappings.append(tuple([el.split("#")[0].replace(".v2", "").rsplit(".",1)[0].replace(".", "_").lower() + "#" + el.split("#")[1] for el in elem]))
data_orig = pickle.load(open("Input/data_webdir.pkl", "rb"))[0]
data_logmap = {}
for key in data_orig:
    data_logmap[key] = False
s = set(list(data_logmap.keys()))
gt_mappings = [tuple(pair) for pair in gt_mappings]
for mapping in gt_mappings:
    
    if mapping in s:
        data_logmap[mapping] = True
    else:
        mapping = tuple([el.replace(",-", "_") for el in mapping])
        if mapping in s:
            data_logmap[mapping] = True
        else:
            print (mapping)
all_metrics = []
def return_test_data(data, i):
    data_t = {elem: data[elem] for elem in data if data[elem]}
    data_f = {elem: data[elem] for elem in data if not data[elem]}

    data_t_items = list(data_t.keys())
    data_f_items = list(data_f.keys())

    test_data_t = data_t_items[int((0.2*i)*len(data_t)):int((0.2*i + 0.2)*len(data_t))]
    test_data_f = data_f_items[int((0.2*i)*len(data_f)):int((0.2*i + 0.2)*len(data_f))]
    
    test_data = {}
    for elem in test_data_t:
        test_data[elem] = True
    for elem in test_data_f:
        test_data[elem] = False
    return test_data

for i in range(5):
    test_gt = return_test_data(data_orig, i)
    test_logmap = {elem: data_logmap[elem] for elem in test_gt}
    tp = len([elem for elem in test_gt if test_gt[elem] and test_logmap[elem]])
    fp = len([elem for elem in test_logmap if not test_gt[elem] and test_logmap[elem]])
    fn = len([elem for elem in test_logmap if test_gt[elem] and not test_logmap[elem]])
    
    try:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1score = 2 * precision * recall / (precision + recall)
        f2score = 5 * precision * recall / (4 * precision + recall)
        f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    except Exception as e:
        print (e)
        continue
    all_metrics.append((precision, recall, f1score, f2score, f0_5score))
    
np.mean(all_metrics, axis=0)

('german_datasets_copy/webdirectory/dmoz.owl', 'german_datasets_copy/webdirectory/google.owl')
('german_datasets_copy/webdirectory/dmoz.owl', 'german_datasets_copy/webdirectory/web.owl')
